## This is the face mask detector.

The following code utilizes Faster R-CNN in order to perform object detection and detec face masks on people in a picture.

### Store image's directory into a list

In [ ]:
#retrieve image imformation
import os
image_directories = []
for dirname, _, filenames in os.walk('archive/images/'):
    for filename in filenames:
        image_directories.append(os.path.join(dirname, filename))

### Create Dataframe based on image's found in XML

In [ ]:
import xml.etree.ElementTree as et
import re
import pandas as pd
import matplotlib.pyplot as plt

dic = {"image": [],"Dimensions": []}
for i in range(1,116):
	dic[f'Object {i}']=[]
print("Generating data in CSV format....")

img_elem = []
img_count = []

for file in os.listdir("archive/annotations"):
    row = []
    xml = et.parse("archive/annotations/"+file) 
    root = xml.getroot()
    
    img = root[1].text
    row.append(img)
    
    num_img = int(img[12:-4])
    img_count.append(num_img)
    
    h,w = root[2][0].text,root[2][1].text
    count = 0
    row.append([h,w])
    temp_img = []
    for i in range(4,len(root)):
        temp = []
        if root[i][0].text != "mask_weared_incorrect":
            
            temp.append(root[i][0].text)
            count +=1
    #         print(root[i][0].text)
            temp_img.append(root[i][0].text) 
            for point in root[i][5]:
                temp.append(point.text)
            row.append(temp)
#     img_elem.append(temp_img)
    img_elem.append(count)
    
    for i in range(len(row),119):
        row.append(0)
    for i,each in enumerate(dic):
        dic[each].append(row[i])
#Main data
df = pd.DataFrame(dic)
#bounding box information for understanding data
df2 = pd.DataFrame(img_elem)

In [ ]:
df

## Cleaning Data

#### Remove pictures with "mask_weared_incorrect"

In [ ]:
#Removes any pictures that formerly only had the "mask_weared_incorrect" label
for i in df.index.values:
    if isinstance(df["Object 1"][i], int):
        df = df.drop(i)

In [ ]:
df2.plot.box()

In [ ]:
#Removes any images containing 40 or more bounding boxes
seriesObj = df2.apply(lambda x: True if x[0] < 40 else False, axis=1)
# print(type(seriesObj))
# print(len(df))

index = seriesObj[seriesObj == False].index.values
# print(list(index))
df = df.drop(index=list(index))
df2 = df2.drop(index=list(index))
# print(len(df))

#Removes extra columns
for i in range (40, 116):
    col_name = "Object "+ str(i)
    df.pop(col_name)

In [ ]:
df2.plot.hist()

### Reformat DF to Bounding Box entry DF

In [ ]:
bb_dict = {"Image": [], "ClassName": [], "XMin": [], "XMax": [], "YMin": [], "YMax": []}

for row in df.index.values:
    j = 0
    for col in df.columns:
        if (j < 2):
            j += 1
            continue
        if (df[col][row] == 0):
            break
        x, y = df["Dimensions"][row]
        x = int(x)
        y = int(y)
        bb_dict["Image"].append(df["image"][row])
        bb_dict["ClassName"].append(df[col][row][0])
        x1 = int(df[col][row][1])
        x2 = int(df[col][row][3])
        y1 = int(df[col][row][2])
        y2 = int(df[col][row][4])
        
        
        bb_dict["XMin"].append(x1 / x)
        bb_dict["XMax"].append(x2 / x)
        bb_dict["YMin"].append(y1 / y)
        bb_dict["YMax"].append(y2 / y)

bb_df = pd.DataFrame(bb_dict)

In [ ]:
bb_df

In [ ]:
bb_df.groupby(bb_df.ClassName == 'with_mask').count()

In [ ]:
bb_df.to_csv('annotate.txt', header=None, index=None, sep=' ')

In [ ]:
from sklearn.model_selection import train_test_split

# train, test = train_test_split(bb_df, test_size=0.2, shuffle=True)
# train, test = train_test_split(df, test_size=0.2)
train_df = bb_df.sample(frac=0.8,random_state=200) #random state is a seed value
test_df = bb_df.drop(train_df.index)

train_df.head()

In [ ]:
test_df.head()

In [ ]:
import imageio as io
import cv2
import matplotlib.pyplot as plt
iou_img_path = 'archive/images/maksssksksss266.png'
img = io.imread(iou_img_path)
xmin = int(0.245847 * img.shape[1])
xmax = int(0.598007 * img.shape[1])
ymin = int(0.512500 * img.shape[0])
ymax = int(0.825000 * img.shape[0])
cv2.rectangle(img, (xmin,ymin), (xmax,ymax), 45, 2)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img, 'dataset label', (xmin,ymin-10), font, 0.5, (0,255,0), 1)

plt.figure(figsize=(15,10))
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
def plot_bbox(img_id):
#     img_url = images_boxable.loc[images_boxable["ImageID"]==img_id]['OriginalURL'].values[0]
#     img = io.imread(img_url)
#     height, width, channel = img.shape
#     print(f"Image: {img.shape}")
#     bboxs = annotations_bbox[annotations_bbox['ImageID']==img_id]

    k = 0

    for idx in bb_dict: 
        if k > 10: 
            break
        k += 1
        xmin = bb_dict['XMin'][idx]
        xmax = bb_dict['XMax'][idx]
        ymin = bb_dict['YMin'][idx]
        ymax = bb_dict['YMax'][idx]
        img_path = bb_dict['Image'][idx]
        img = cv2.imread('archive/images/' + str(img_path))
        
        x = img.shape[0]
        y = img.shape[1] 

### Padding

In [ ]:
def plot_bbox(img_id):
#     img_url = images_boxable.loc[images_boxable["ImageID"]==img_id]['OriginalURL'].values[0]
#     img = io.imread(img_url)
#     height, width, channel = img.shape
#     print(f"Image: {img.shape}")
#     bboxs = annotations_bbox[annotations_bbox['ImageID']==img_id]

    k = 0

    for idx in bb_dict: 
        if k > 10: 
            break
        k += 1
        xmin = bb_dict['XMin'][idx]
        xmax = bb_dict['XMax'][idx]
        ymin = bb_dict['YMin'][idx]
        ymax = bb_dict['YMax'][idx]
        img_path = bb_dict['Image'][idx]
        img = cv2.imread('archive/images/' + str(img_path))
        
        x = img.shape[0]
        y = img.shape[1] 

In [ ]:
df

In [ ]:
max_x = 0
max_y = 0
lengths = []
widths = []
for index in df.index.values:
    max_x = int(df["Dimensions"][index][0]) if int(df["Dimensions"][index][0]) > max_x else max_x
    lengths.append(int(df["Dimensions"][index][0]))
    max_y = int(df["Dimensions"][index][1]) if int(df["Dimensions"][index][1]) > max_y else max_y
    widths.append(int(df["Dimensions"][index][1]))
print(max_x, max_y)

In [ ]:
import cv2
import matplotlib.image as mplib
import matplotlib.pyplot as plt

#This includes ideal padding for images, however we need to find a way to adjust each bounding box accordingly 
for i in df.index.values:
    img = mplib.imread(image_directories[i])
#     print("before: ", img)
    Y, X = df["Dimensions"][i]
#     print(max_x-int(X), max_y-int(Y))
    new_img = img
    if (max_y-int(Y)) <  (max_x-int(X)):
        new_x = max_y - int(Y)+int(X)
        new_img = cv2.resize(img, (new_x, max_y))
#         print(new_x)
        new_img = cv2.copyMakeBorder(new_img, 0, 0, 0, max_x - new_x, cv2.BORDER_CONSTANT)
    elif (max_y-int(Y)) >  (max_x-int(X)):
        new_y = max_x - int(X)+int(Y)
        new_img = cv2.resize(img, (max_x, new_y))
        new_img = cv2.copyMakeBorder(new_img, 0, max_y-new_y, 0, 0, cv2.BORDER_CONSTANT)
#     print(max_x-(max_y - int(X)+int(Y)))
#     if max_x-(max_y - int(X)+int(Y)) > 0:
#         new_img = cv2.copyMakeBorder(img, 0, 0, 0,max_x-(max_y - int(X)+int(Y)), cv2.BORDER_CONSTANT)
#     else:
#         new_img = cv2.copyMakeBorder(img, 0, max_y - max_x - , 0, max_x - max_y-int(X), cv2.BORDER_CONSTANT)
#     plt.imshow(mpl_img)

In [ ]:
print(new_img.shape)
plt.imshow(new_img)

In [ ]:
import matplotlib.patches as patches

img_arr = []

for i in df.index.values:
#     print(image_directories[i])
    img = mplib.imread(os.path.join("archive/images/", df["image"][i]))
    fig,ax = plt.subplots(1)

    # Display the image
    
    
#     print("before: ", img)
    X, Y = df["Dimensions"][i]
    img = cv2.copyMakeBorder(img, 0, max_y - int(Y), 0, max_x-int(X), cv2.BORDER_CONSTANT)
    print(i)
    ax.imshow(img)
    first_box = df["Object 1"][i]
    if isinstance(first_box, list) :
#         print(first_box[1])
        x_min = first_box[1]
        y_min = first_box[2]
        x_max = first_box[3]
        y_max = first_box[4]
        rect = patches.Rectangle((int(x_min),int(y_min)),
                                 int(x_max)-int(x_min),
                                 int(y_max)-int(y_min),
                                 linewidth=1,edgecolor='r',facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)
        img_arr.append(img)

In [ ]:
img.shape

In [ ]:
import numpy as np

variances = np.array(v)
print(v)
fig, ax = plt.subplots(figsize =(10, 7))
ax.plot(range(1, 40, 3), v, marker='o')
ax.set_xlabel("Principle Components")
ax.set_ylabel("Variance")
ax.set_title("Variance for Various Principle Component Values in PCA")

### PCA

In [ ]:
reshaped_img = np.reshape(img_arr[3], (img.shape[0], img.shape[1]*img.shape[2]))
        
pca = PCA(18).fit(reshaped_img) 
img_transformed = pca.transform(reshaped_img)
print(np.sum(pca.explained_variance_ratio_))

temp = pca.inverse_transform(img_transformed) 
print(temp.shape)
temp = np.reshape(temp, (600,600 ,4)) 
print(temp.shape) 
plt.imshow(temp)

In [ ]:
plt.imshow(img)

In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import math
import cv2
import copy
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import os

from sklearn.metrics import average_precision_score

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed
from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.objectives import categorical_crossentropy

from keras.models import Model
from keras.utils import generic_utils
from keras.engine import Layer, InputSpec
from keras import initializers, regularizers

Using TensorFlow backend.


In [ ]:
def img_output_length(width, height):
    return width//16, height//16    

def nn_base(tensor=None, trainable=False):

    input_shape = (None, None, 3)

    if not K.is_keras_tensor(input_tensor):
        img_input = Input(tensor=input_tensor, shape=input_shape)
    else:
        img_input = tensor

    r = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    r = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(r)
    r = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(r)
    r = Confdov2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(r)
    r = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(r)
    r = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(r)
    r = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(r)
    r = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(r)
    r = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(r)
    r = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(r)
    r = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(r)
    r = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(r)
    r = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(r)
    r = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(r)
    r = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(r)
    r = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(r)
    r = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(r)
    return r

### Classifier

In [ ]:
def classifier(lyrs, i_rois, nums, nb = 4):
    pool = RoiPoolingConv(7, nums)([lyrs, i_rois])
    k = TimeDistributed(Flatten(name='flatten'))(pool)
    l = TimeDistributed(Dense(4096, activation='relu', name='fc1'))(l)
    l = TimeDistributed(Dropout(0.5))(l)
    l = TimeDistributed(Dense(4096, activation='relu', name='fc2'))(l)
    l = TimeDistributed(Dropout(0.5))(l)
    l_cl = TimeDistributed(Dense(nb_classes, activation='softmax', kernel_initializer='zero'), name='dense_class_{}'.format(nb))(l)
    l_r = TimeDistributed(Dense(4 * (nb_classes-1), activation='linear', kernel_initializer='zero'), name='dense_regress_{}'.format(nb))(l)
    return [l_cl, lr]

### Converting RPN Layer to ROI 

In [ ]:
def rpn_to_roi(rpn_layer, regr_layer, C, dim_ordering, use_regr=True, max_boxes=300,overlap_thresh=0.9):

    regr_layer = regr_layer / C.std_scaling

    rows, cols, _ = rpn_layer.shape
    A = np.zeros((4, rpn_layer.shape[1], rpn_layer.shape[2], rpn_layer.shape[3]))

    for anchor_size in C.anchor_box_scales:
        for anchor_ratio in C.anchor_box_ratios:
            anchor_x = (anchor_size * anchor_ratio[0])/C.rpn_stride
            anchor_y = (anchor_size * anchor_ratio[1])/C.rpn_stride
            regr = regr_layer[0, :, :, 4 * curr_layer:4 * curr_layer + 4] 
            regr = np.transpose(regr, (2, 0, 1)) 
            X, Y = np.meshgrid(np.arange(cols),np. arange(rows))
            A[0, :, :, curr_layer] = X - anchor_x/2 
            A[1, :, :, curr_layer] = Y - anchor_y/2 
            A[2, :, :, curr_layer] = anchor_x       
            A[3, :, :, curr_layer] = anchor_y       
            if use_regr:
                A[:, :, :, curr_layer] = apply_regr_np(A[:, :, :, curr_layer], regr)
            A[2, :, :, curr_layer] = np.maximum(1, A[2, :, :, curr_layer])
            A[3, :, :, curr_layer] = np.maximum(1, A[3, :, :, curr_layer])
            A[2, :, :, curr_layer] += A[0, :, :, curr_layer]
            A[3, :, :, curr_layer] += A[1, :, :, curr_layer]
            A[0, :, :, curr_layer] = np.maximum(0, A[0, :, :, curr_layer])
            A[1, :, :, curr_layer] = np.maximum(0, A[1, :, :, curr_layer])
            A[2, :, :, curr_layer] = np.minimum(cols-1, A[2, :, :, curr_layer])
            A[3, :, :, curr_layer] = np.minimum(rows-1, A[3, :, :, curr_layer])
            curr_layer += 1
    all_boxes = np.reshape(A.transpose((0, 3, 1, 2)), (4, -1)).transpose((1, 0)) 
    all_probs = rpn_layer.transpose((0, 3, 1, 2)).reshape((-1))                   
    x1 = all_boxes[:, 0]
    y1 = all_boxes[:, 1]
    x2 = all_boxes[:, 2]
    y2 = all_boxes[:, 3]
    idxs = np.where((x1 - x2 >= 0) | (y1 - y2 >= 0))
    all_boxes = np.delete(all_boxes, idxs, 0)
    all_probs = np.delete(all_probs, idxs, 0)
    result = non_max_suppression_fast(all_boxes, all_probs, overlap_thresh=overlap_thresh, max_boxes=max_boxes)[0]

    return result

### Calculating the IoU

In [ ]:
def intersection_over_union(first, second):

    if first[0] >= first[2] || first[1] >= first[3] || second[0] >= second[2] | second[1] >= second[3]:
        return 0

    area_i = intersection(first, second)
    area_u = union(first, second, area_i)

    return float(area_i) / float(area_u + 1e-6)


def union(first, second, area_intersection):
    area_a = (first[3] - first[1]) * (first[2] - first[0])
    area_b = (second[3] - second[1]) * (second[2] - second[0])
    
    return area_a + area_b - area_intersection


def intersection(first, second):
    
    x = max(first[0], second[0])
    y = max(first[1], second[1])
    
    width = min(first[2], second[2]) - x
    height = min(first[3], second[3]) - y
    

    if height < 0 || width < 0:
        return 0
    
    return width * height